
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::484830312924:role/Glue-admin-role-for-RS
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 9b2423b5-160d-4eff-a1c6-e620a939965c
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session 9b2423b5-160d-4eff-a1c6-e620a939965c to get into ready status...
Session 9b2423b5-160d-4eff-a1c6-e620a939965c has been created.



## Glue DynamicFrame

### Read data from Redshift, Write data to Redshift

#### In the previous Notebook we saw how to create DynamicFrame from reading source data from S3 using the catalog
#### In this Notebook we will see how to read and write data from and to Redshift, create DynamicFrames, convert to DataFrame and use Spark SQL to perform ETL operations

#### __Variable naming: All DynamicFrames end with "DyF" and all DataFrames ends with "DF"__

In [2]:
my_conn_options = {  
    "url": "jdbc:redshift://<cluster-name>.c8qeuccnncmq.us-east-1.redshift.amazonaws.com:5439/retail",
    "dbtable": "sales",
    "user": "awsuser",
    "password": "<pwd>",
    "redshiftTmpDir": "s3://<path>",
    "aws_iam_role": "<role-arn>"
}

salesDyF = glueContext.create_dynamic_frame_from_options("redshift", my_conn_options)

In [3]:
salesDyF.show(10)

{"invoice_no": "540680", "product_id": "21245", "quantity": "2", "invoice_date": "1/10/11 16:24", "unit_price": "1.69", "customer_id": "15311", "country": "United Kingdom"}
{"invoice_no": "541592", "product_id": "21051", "quantity": "3", "invoice_date": "1/19/11 15:08", "unit_price": "1.63", "customer_id": "17850", "country": "United Kingdom"}
{"invoice_no": "537641", "product_id": "22516", "quantity": "1", "invoice_date": "12/7/10 15:32", "unit_price": "4.21", "customer_id": "17591", "country": "United Kingdom"}
{"invoice_no": "537422", "product_id": "22791", "quantity": "12", "invoice_date": "12/6/10 15:27", "unit_price": "1.25", "customer_id": "17812", "country": "United Kingdom"}
{"invoice_no": "C537431", "product_id": "21755", "quantity": "-1", "invoice_date": "12/6/10 16:10", "unit_price": "5.95", "customer_id": "16782", "country": "United Kingdom"}
{"invoice_no": "549734", "product_id": "22089", "quantity": "2", "invoice_date": "4/11/11 17:12", "unit_price": "2.95", "customer_id

In [4]:
my_conn_options = {  
    "url": "jdbc:redshift://<cluster-name>.c8qeuccnncmq.us-east-1.redshift.amazonaws.com:5439/retail",
    "dbtable": "product",
    "user": "awsuser",
    "password": "<owd>",
    "redshiftTmpDir": "s3://<path>",
    "aws_iam_role": "<role-arn>"
}

productDyF = glueContext.create_dynamic_frame_from_options("redshift", my_conn_options)

In [5]:
productDyF.show(10)

{"product_id": "23316", "product_desc": "GLASS APOTHECARY BOTTLE PERFUME"}
{"product_id": "22717", "product_desc": "FRENCH BLUE METAL DOOR SIGN 1"}
{"product_id": "37423", "product_desc": "Adjustment"}
{"product_id": "22906", "product_desc": "12 MESSAGE CARDS WITH ENVELOPES"}
{"product_id": "22071", "product_desc": "SMALL WHITE RETROSPOT MUG IN BOX"}
{"product_id": "23056", "product_desc": "Display"}
{"product_id": "DCGS0072", "product_desc": "15 PINK FLUFFY CHICKS IN BOX"}
{"product_id": "85111", "product_desc": "SET OF 6 HEART CHOPSTICKS"}
{"product_id": "21246", "product_desc": "DOORMAT MULTICOLOUR STRIPE"}
{"product_id": "23519", "product_desc": "SWEETHEART 3 TIER CAKE STAND"}
{"product_id": "84789", "product_desc": "GOLD M PEARL  ORBIT NECKLACE"}
{"product_id": "20892", "product_desc": "NUMBER TILE VINTAGE FONT 4"}
{"product_id": "90024C", "product_desc": "ANT COPPER TURQ BOUDICCA BRACELET"}
{"product_id": "22249", "product_desc": "GOLD DIAMANTE STAR BROOCH"}
{"product_id": "84576

In [6]:
my_conn_options = {  
    "url": "jdbc:redshift://<cluster-name>.c8qeuccnncmq.us-east-1.redshift.amazonaws.com:5439/retail",
    "dbtable": "customer",
    "user": "awsuser",
    "password": "<pwd>",
    "redshiftTmpDir": "<path>",
    "aws_iam_role": "<role-arn>n"
}

customerDyF = glueContext.create_dynamic_frame_from_options("redshift", my_conn_options)

In [7]:
customerDyF.show(10)

{"customer_id": 13243, "title": "Mr.", "first_name": "Jessie", "middle_name": "E.", "last_name": "Valerio", "company_name": "Wire Baskets and Parts", "email_address": "jessie0@adventure-works.com", "phone_no": "103-555-0179", "join_date": "8/1/05 0:00"}
{"customer_id": 13336, "title": "Mr.", "first_name": "Robert", "middle_name": "L.", "last_name": "Barker", "company_name": "Kickstand Sellers", "email_address": "robert3@adventure-works.com", "phone_no": "241-555-0112", "join_date": "12/1/06 0:00"}
{"customer_id": 16120, "title": "Mr.", "first_name": "Gregory", "middle_name": "T.", "last_name": "Vanderbout", "company_name": "Convenient Sales and Service", "email_address": "gregory1@adventure-works.com", "phone_no": "684-555-0134", "join_date": "9/1/06 0:00"}
{"customer_id": 15221, "title": "Mr.", "first_name": "Shaun", "middle_name": null, "last_name": "Beasley", "company_name": "Finer Cycle Shop", "email_address": "shaun0@adventure-works.com", "phone_no": "396-555-0187", "join_date": "

In [9]:
# We will perform just the join and skip the rest of the DynamicFrame operations, please refer to the other Notebook "AWS Glue Notebook with PySpark.ipynb" if you want to explore DynamicFrame functions on these datasets

joinedDyF = Join.apply(customerDyF, Join.apply(salesDyF, productDyF, "product_id", "product_id"), "customer_id", "customer_id")

In [10]:
# As the joined dyanmic frame will have wider columns from all the 3 dynamic frames, we will use a DynamicFrame class called "select_fields" to display only a set of selective fileds and also conver to a DataFrame

selectColumnsDF = joinedDyF.select_fields(paths=["customer_id", "first_name", "last_name", "product_id", "product_desc", "quantity", "unit_price", "country", "invoice_no"]).toDF()

In [22]:
# Add a new derived column to the DataFrame, let's call the new dervied column as "total_sales" which is derived by multipying quantity with unit_price for each transaction
from pyspark.sql.functions import col

derivedColDF = selectColumnsDF.withColumn("total_sales", col("quantity").cast("Integer") * col("unit_price").cast("Float"))
derivedColDF.show(10)

+-----------+----------+---------+----------+--------------------+--------+----------+--------------+----------+-----------+
|customer_id|first_name|last_name|product_id|        product_desc|quantity|unit_price|       country|invoice_no|total_sales|
+-----------+----------+---------+----------+--------------------+--------+----------+--------------+----------+-----------+
|      13835|      Todd|    Logan|     21888|           BINGO SET|       4|      3.75|United Kingdom|    552309|       15.0|
|      13835|      Todd|    Logan|     22464|HANGING MEDINA LA...|      12|      1.65|United Kingdom|    552309|       19.8|
|      13835|      Todd|    Logan|     21936| KNITTED RABBIT DOLL|       5|      2.95|United Kingdom|    552309|      14.75|
|      13835|      Todd|    Logan|     21936| KNITTED RABBIT DOLL|      10|      2.95|United Kingdom|    559326|       29.5|
|      13835|      Todd|    Logan|     22728|ALARM CLOCK BAKEL...|       4|      3.75|United Kingdom|    571657|       15.0|


## DataFrame, Spark SQL

In [23]:
# We will explore Spark SQL on this newly created DataFrame, for that we will have create  a temp veiw

derivedColDF.createOrReplaceTempView("salesDF")

### Select, Where Clause

In [24]:
# Using Spark SQL select data from the temprory view which is nothing but teh DataFrame that we created in the previous steps
# Note that with Spark SQL, we are writing the ANSI SQL that we use in the traditional databases

spark.sql("SELECT * FROM salesDF LIMIT 10").show(truncate=False)

+-----------+----------+---------+----------+-----------------------------------+--------+----------+--------------+----------+-----------+
|customer_id|first_name|last_name|product_id|product_desc                       |quantity|unit_price|country       |invoice_no|total_sales|
+-----------+----------+---------+----------+-----------------------------------+--------+----------+--------------+----------+-----------+
|13835      |Todd      |Logan    |21888     |BINGO SET                          |4       |3.75      |United Kingdom|552309    |15.0       |
|13835      |Todd      |Logan    |22464     |HANGING MEDINA LANTERN SMALL       |12      |1.65      |United Kingdom|552309    |19.8       |
|13835      |Todd      |Logan    |21936     |KNITTED RABBIT DOLL                |5       |2.95      |United Kingdom|552309    |14.75      |
|13835      |Todd      |Logan    |21936     |KNITTED RABBIT DOLL                |10      |2.95      |United Kingdom|559326    |29.5       |
|13835      |Todd   

In [25]:
# Where clause to filter data that belongs to Spain

spark.sql("SELECT * FROM salesDF WHERE country = 'Spain' LIMIT 10").show(truncate=False)

+-----------+----------+---------+----------+-----------------------------------+--------+----------+-------+----------+-----------+
|customer_id|first_name|last_name|product_id|product_desc                       |quantity|unit_price|country|invoice_no|total_sales|
+-----------+----------+---------+----------+-----------------------------------+--------+----------+-------+----------+-----------+
|12462      |Valerie   |Hendricks|21078     |DECORATION  BUTTERFLY  MAGIC GARDEN|12      |0.85      |Spain  |543541    |10.200001  |
|12462      |Valerie   |Hendricks|84817     |NUMBER TILE VINTAGE FONT 9         |-4      |2.1       |Spain  |C581148   |-8.4       |
|12462      |Valerie   |Hendricks|84817     |NUMBER TILE VINTAGE FONT 9         |6       |2.1       |Spain  |577606    |12.599999  |
|12462      |Valerie   |Hendricks|37450     |LOVE HEART POCKET WARMER           |6       |2.95      |Spain  |543541    |17.7       |
|12462      |Valerie   |Hendricks|37450     |LOVE HEART POCKET WARMER

### Aggregation

In [29]:
# Spark provides powerful functions using which we can efficiently perform complex data analytics like aggregations, joins etc
# With the DataFrame, we will perform data analysis by using aggregate function to see total sales by Customer, total sales by Customer by Product, total revenue by country.
# We will use spark SQL to perform these aggregate functions

#Total sales by Customer, alias aggregate column to a new name, order sum_sales descending with the highest sales on the top. Round sum_sales to 2 decimal places

spark.sql("SELECT customer_id, round(sum(total_sales),2) as sum_sales FROM salesDF GROUP BY customer_id ORDER BY sum_sales DESC").show(truncate=False)


+-----------+------------------+
|customer_id|         sum_sales|
+-----------+------------------+
|      17850|1217571.9390883408|
|      14646| 280549.8192884624|
|      18102|256438.48995232582|
|      17450|200736.01051712036|
|      14911| 137108.4096262157|
|      12415|123725.44990730286|
|      17511|122776.22992050648|
|      14156|121753.16940319538|
|      15311| 85180.03940740228|
|      16684| 65892.07912826538|
|      13694| 63016.29979014397|
|      13089| 57855.68000161648|
|      14096|  57201.9700319469|
|      15061|54228.739621818066|
|      17949|52750.839977264404|
|      15769|  51850.1201082468|
|      16029| 50992.60983848572|
|      14298| 50988.22000473738|
|      14088| 50537.88961625099|
|      17841|46063.909761846066|
+-----------+------------------+
only showing top 20 rows


In [32]:
# Total sales by Customer by Product order by sum_sales with the highest sale on the top. Round sum_sales to 2 decimal places

spark.sql("SELECT customer_id, product_id, round(sum(total_sales),2) as sum_sales FROM salesDF GROUP BY customer_id, product_id ORDER BY sum_sales DESC").show(truncate=False)

+-----------+----------+---------+
|customer_id|product_id|sum_sales|
+-----------+----------+---------+
|17850      |DOT       |167802.89|
|15098      |22502     |39619.5  |
|17850      |47566     |30151.46 |
|17450      |85123A    |26507.68 |
|17850      |22423     |23338.07 |
|17850      |22355     |20109.58 |
|17850      |22086     |18518.56 |
|18102      |21623     |16592.08 |
|16210      |21137     |16027.92 |
|18102      |82484     |15351.0  |
|17850      |23084     |15224.36 |
|18102      |22189     |14089.9  |
|15061      |22423     |14081.7  |
|17450      |22470     |13427.64 |
|17850      |22197     |13238.77 |
|17850      |22114     |12834.78 |
|14096      |DOT       |11906.36 |
|17850      |20724     |10756.08 |
|17450      |23243     |10566.73 |
|17450      |22469     |10201.98 |
+-----------+----------+---------+
only showing top 20 rows


In [33]:
# Total sales by Country

spark.sql("SELECT country, round(sum(total_sales),2) as sum_sales FROM salesDF GROUP BY country ORDER BY sum_sales DESC").show(truncate=False)

+---------------+----------+
|country        |sum_sales |
+---------------+----------+
|United Kingdom |8246600.42|
|Netherlands    |285828.54 |
|EIRE           |266116.74 |
|Germany        |221675.73 |
|France         |197535.65 |
|Australia      |137094.97 |
|Switzerland    |56550.95  |
|Spain          |54926.23  |
|Belgium        |40893.08  |
|Sweden         |36595.91  |
|Japan          |35340.62  |
|Norway         |35163.46  |
|Portugal       |29417.24  |
|Finland        |22397.54  |
|Channel Islands|20404.89  |
|Denmark        |18768.14  |
|Italy          |17015.46  |
|Cyprus         |13332.69  |
|Austria        |10154.32  |
|Hong Kong      |10117.04  |
+---------------+----------+
only showing top 20 rows


In [35]:
# Total sales by customer, Product, Country

spark.sql("SELECT customer_id, product_id, country, round(sum(total_sales),2) as sum_sales FROM salesDF GROUP BY customer_id, product_id, country ORDER BY sum_sales DESC").show(truncate=False)

+-----------+----------+--------------+---------+
|customer_id|product_id|country       |sum_sales|
+-----------+----------+--------------+---------+
|17850      |DOT       |United Kingdom|167802.89|
|15098      |22502     |United Kingdom|39619.5  |
|17850      |47566     |United Kingdom|30151.46 |
|17450      |85123A    |United Kingdom|26507.68 |
|17850      |22423     |United Kingdom|23338.07 |
|17850      |22355     |United Kingdom|20109.58 |
|17850      |22086     |United Kingdom|18518.56 |
|18102      |21623     |United Kingdom|16592.08 |
|16210      |21137     |United Kingdom|16027.92 |
|18102      |82484     |United Kingdom|15351.0  |
|17850      |23084     |United Kingdom|15224.36 |
|18102      |22189     |United Kingdom|14089.9  |
|15061      |22423     |United Kingdom|14081.7  |
|17450      |22470     |United Kingdom|13427.64 |
|17850      |22197     |United Kingdom|13238.77 |
|17850      |22114     |United Kingdom|12834.78 |
|14096      |DOT       |United Kingdom|11906.36 |


#### You can play around with the multitude of powerful functions that the Spark SQL and DataFrame provides, have shown few examples so you can follow and understand the concept. 
#### The main focus of this Notebook is to demonstrate how to Read and Write from and to Redshift using Glue Notebook and perfom ETL on the DataFrame using Spark SQL

### Write data to Redshift

In [42]:
# For simplicity and to write to Glue Data Catalog, we can convert the Spark DataFrame back to Glue DynamicFrame and write to either Redshift directly or to Catalog
# Convert back to DynamicFrame
from awsglue.dynamicframe import DynamicFrame

derivedColDyF = DynamicFrame.fromDF(derivedColDF,glueContext, "convert")

In [46]:
my_conn_options = {
    "dbtable": "sales_tgt",
    "database": "retail",
    "aws_iam_role": "<role-arn>"
}

glueContext.write_dynamic_frame.from_jdbc_conf(
    frame = derivedColDyF, 
    catalog_connection = "Redshift-Direct", 
    connection_options = my_conn_options, 
    redshift_tmp_dir = "<path>")